# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Построить модель со значением метрики качества F1 не меньше 0.75.

Инструкция по выполнению проекта

* Загрузить и подготовьть данные.
* Обучить разные модели.
* Сделать выводы.
* Для выполнения проекта применять BERT необязательно.

Описание данных:

Данные находятся в файле toxic_comments.csv. Столбец text в нём содержит текст комментария, а toxic — целевой признак.

## 1. Подготовка

In [ ]:
!/opt/conda/bin/python -m pip install wordcloud #дополнительные установщики

In [ ]:
!pip install --upgrade Pillow #дополнительные установщики

In [ ]:
!pip install spacy #дополнительные установщики

In [ ]:
!pip install transformers

In [ ]:
!pip install imblearn

In [ ]:
!pip install sklearn

In [ ]:
!pip install --upgrade scikit-learn

In [ ]:
!pip install pandarallel

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
from pandarallel import pandarallel #импорт дополнительных библиотек  
pandarallel.initialize(progress_bar = True) #импорт дополнительных библиотек
from tqdm import tqdm
tqdm.pandas(desc="progress") #импорт дополнительных библиотек
import nltk
nltk.download('wordnet') #без этих пакетов программа не выполнялась на локалке
nltk.download('averaged_perceptron_tagger') #без этих пакетов программа не выполнялась на локалке
nltk.download('omw-1.4') #без этих пакетов программа не выполнялась на локалке
nltk.download('punkt') #без этих пакетов программа не выполнялась на локалке
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize #импорт дополнительных библиотек
from nltk.corpus import stopwords as stopwords
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, roc_auc_score, roc_curve, mean_squared_error, make_scorer
from sklearn.utils import shuffle
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV #импорт дополнительных библиотек
from imblearn.pipeline import Pipeline as imbPipeline #импорт дополнительных библиотек
from imblearn.over_sampling import RandomOverSampler #импорт дополнительных библиотек
from imblearn.under_sampling import RandomUnderSampler #импорт дополнительных библиотек
import seaborn as sns
import matplotlib.pyplot as plt
import transformers #без этих пакетов программа не выполнялась на локалке
import warnings
warnings.filterwarnings('ignore')

In [ ]:
try:
    toxic_comments = pd.read_csv('/datasets/toxic_comments.csv', index_col=0)
except:
    toxic_comments = pd.read_csv('C:\\Data\\toxic_comments.csv', index_col=0)

In [ ]:
toxic_comments.info()

In [ ]:
toxic_comments.head(10)

In [ ]:
#toxic_comments = toxic_comments.applymap(lambda x: x.lower() if isinstance(x, str) else x)

In [ ]:
print(toxic_comments.head(10))

In [ ]:
toxic_comments.sample()

In [ ]:
toxic_comments.describe()

In [ ]:
toxic_comments.isna().sum()

In [ ]:
toxic_comments.duplicated().sum()

In [ ]:
zeroes = toxic_comments['toxic'].value_counts()[0]
ones = toxic_comments['toxic'].value_counts()[1]

#Посмотрим количество токсичных комментариев
print(toxic_comments['toxic'].value_counts())

In [ ]:
plt.figure(figsize=(15, 6))
sns.countplot(x='toxic', data=toxic_comments)
plt.show()

Видим большой дисбаланс классов

In [ ]:
corpus = toxic_comments['text'].values
print(corpus)

In [ ]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [ ]:
lemmatizer = WordNetLemmatizer()

#def clear_text(text):
#    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
#    clear = pattern.split()
#    lemm = []
#    for i in range(len(clear)):
#        lemm.append(lemmatizer.lemmatize(clear[i]))
#    return " ".join(lemm)

In [ ]:
#%%time

#for i in tqdm(range(len(corpus))):
#    corpus[i] = clear_text(corpus[i]) 

In [ ]:
def lemmatize_text(text):
    lemmatizer = nltk.WordNetLemmatizer()
    lemmatized_tokens = []
    for token in nltk.word_tokenize(text):
        lemmatized_token = lemmatizer.lemmatize(token, get_wordnet_pos(token))
        lemmatized_tokens.append(lemmatized_token)
    return ' '.join(lemmatized_tokens)

In [ ]:
%%time

toxic_comments['lemm_text'] = toxic_comments['text'].apply(lambda x: lemmatize_text(x))

In [ ]:
sentence1 = "The striped bats are hanging on their feet for best"
sentence2 = "you should be ashamed of yourself went worked"
df_my = pd.DataFrame([sentence1, sentence2], columns = ['text'])
print(df_my)

In [ ]:
print(df_my['text'].apply(lemmatize_text))

In [ ]:
def get_preprocessed_text(text):
    text = text.lower()
    text = word_tokenize(text)
    from nltk.tokenize import word_tokenize
    from nltk.corpus import wordnet
    from nltk.corpus import stopwords
    from nltk.stem.wordnet import WordNetLemmatizer
    tag = nltk.pos_tag([token])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
    lemmatizer = WordNetLemmatizer()
    # применяем лемматизацию, используя теги
    text = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in text]

    # инициируем множество стоп-слов
    stop_words = set(stopwords.words('english'))
    # удаляем стоп-слова из документа
    text = [token for token in text if token not in stop_words]

    # возвращаем результат
    return text

In [ ]:
toxic_comments['lemmatized_paralel'] = toxic_comments['text'].parallel_apply(get_preprocessed_text) 

In [ ]:
document = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in document]

In [ ]:
stop_words = set(stopwords.words('english'))

document = [token for token in document if token not in stop_words]

In [ ]:
toxic_comments_corpus = pd.DataFrame(corpus)
toxic_comments['lemm_text'] = toxic_comments_corpus
display(toxic_comments.head(10))
toxic_comments.info()

In [ ]:
#column_name = 'Unnamed: 0' #Удалим ненужный столбец
#toxic_comments = toxic_comments.drop(column_name, axis=1)

In [ ]:
display(toxic_comments)

Сделаем стоп-слова и векторизацию текста

In [ ]:
toxic_comments['lemm_text'].isna().sum()

In [ ]:
toxic_comments['toxic'].isna().sum()

In [ ]:
toxic_comments = toxic_comments.dropna(subset=['lemm_text'])

In [ ]:
print(toxic_comments['lemm_text'].isna().sum())

In [ ]:
#try:
#    nltk.download('stopwords')
#except:
#    pass

In [ ]:
features = toxic_comments['lemm_text']
target = toxic_comments['toxic']

features_train_1, features_test_1, target_train, target_test = train_test_split(
features, target, test_size=0.25, random_state=12345, stratify=target)

#Выведем размер выборок
display(features_train_1.shape)
display(features_test_1.shape)
display(target_train.shape)
display(target_test.shape)

In [ ]:
# Загрузка стоп-слов из NLTK
try:
    nltk.download('stopwords')
except:
    pass

# Создание списка стоп-слов
try:
    stop_words = stopwords.words("english")
except:
    pass

# Создание экземпляра TfidfVectorizer с передачей списка стоп-слов
count_tf_idf = TfidfVectorizer(stop_words=stop_words)

features_train = count_tf_idf.fit_transform(features_train_1)


In [ ]:
features_test = count_tf_idf.transform(features_test_1)

In [ ]:
toxic_comments.isna().sum()

Определим целевой признак и делим выборку на тренировочную и тестовую

In [ ]:
print(type(features_train))
print(type(features_train[0]))

In [ ]:
%%time

pipeline_oversampling = imbPipeline([
    ('tfidf', TfidfVectorizer()),
    ('oversampling', RandomOverSampler(random_state=0)),
    ('logreg', LogisticRegression(random_state=42))])


parameters = {
    'tfidf__max_df': (0.25, 0.5, 0.75),
    'tfidf__ngram_range': [(1, 1), (1, 2), (1, 3)],
    'logreg__C': [1,2,6]
}

grid_search_tune = HalvingGridSearchCV(pipeline_oversampling, parameters, cv=3, n_jobs=-1, scoring='f1', verbose=3)
grid_search_tune.fit(features_train, target_train)

print(f"Best score: {abs(grid_search_tune.best_score_)}, Best params: {grid_search_tune.best_params_}")

In [ ]:
print(features_train.shape)
print(features_test.shape)
print(target_train.shape)
print(target_test.shape)

**Вывод:**
В ходе загрузки и подготовки данных к обучению мы их загрузили, проверили на пропуски и дубликаты, а также удалили лишний столбец `Unnamed: 0`.

Мы выделили целевой признак `lemm_text`. Также мы использовали TfidfVectorizer для векторизации текстов.

Затем мы разделили данные на обучающую и тестовую выборки.

## 2. Обучение

In [ ]:
const_model = [1 for i in range(len(target_test))]
const_f1 = f1_score(target_test.reset_index(drop=True), const_model)
print(const_f1)

В обучении мы будем использовать три модели: Логистическую регрессию, случайный лес и классификатор CatBoost.

In [ ]:
#def RMSE(target, predict):
#    return (mean_squared_error(target, predict))**0.5
#RMSE_score = make_scorer(RMSE, greater_is_better=False)

**Логистическая регрессия**

In [ ]:
%%time

model_lr = LogisticRegression(fit_intercept=True, 
                                class_weight='balanced', 
                                random_state=42,
                                solver='liblinear')

#Определяем словарь с набором параметров
regression_parameters = {'C': [0.1, 1, 10]}

#Применяем GridSearchCV с кросс-валидацией
regression_grid = GridSearchCV(model_lr, regression_parameters, scoring='f1', cv=3, error_score='raise')
regression_grid.fit(features_train, target_train)

model_lr.fit(features_train, target_train)
lr_cv_score = pd.Series(cross_val_score(model_lr, features_train, target_train, scoring='f1', cv=3)).mean()
print('Среднее качество модели логистической регрессии на кросс-валидации', lr_cv_score)

Определение оптимальных гиперпараметров для кросс-валидации

In [ ]:
lr_params = regression_grid.best_params_
lr_score = regression_grid.score(features_train, target_train)
print(lr_params)
print(lr_score)

**Случайный лес**

In [ ]:
%%time

model_rf = RandomForestClassifier(class_weight='balanced', n_jobs=-1)

rf_parameters = {
    'n_estimators': range(20, 40, 5),
    'max_depth': range(4, 8, 2),
    'min_samples_leaf': range(3,5),
    'min_samples_split': range(2,6,2),
}

rf_grid = GridSearchCV(model_rf, rf_parameters, scoring='f1', cv=3)
rf_grid.fit(features_train, target_train)

model_rf.fit(features_train, target_train)
rf_cv_score = pd.Series(cross_val_score(model_rf, features_train, target_train, scoring='f1', cv=3)).mean()
print('Среднее качество модели случайного леса на кросс-валидации', rf_cv_score)

Определение оптимальных гиперпараметров для кросс-валидации

In [ ]:
rf_params = rf_grid.best_params_
rf_score = rf_grid.score(features_train, target_train)
print(rf_params)
print(rf_score)

**Catboost**

In [ ]:
features_train_cb = features_train_1[:900]
features_test_cb = features_test_1[:300]
target_train_cb = target_train[:900]
target_test_cb = target_test[:300]

In [ ]:
features_train_cb = count_tf_idf.fit_transform(features_train_cb)
features_test_cb = count_tf_idf.transform(features_test_cb)

In [ ]:
model_cb = CatBoostClassifier(class_weights=[1, zeroes/ones], iterations=30)

cb_parametrs = {'depth': [4, 8]}

cb_grid = GridSearchCV(model_cb, cb_parametrs, scoring='f1', cv=3)
cb_grid.fit(features_train_cb, target_train_cb, verbose=10)
cb_cv_score = pd.Series(cross_val_score(model_cb, features_train, target_train, scoring='f1', cv=3)).mean()
print('Среднее качество Catboost на кросс-валидации', cb_cv_score)

Определение оптимальных гиперпараметров для кросс-валидации

In [ ]:
%%time

cb_params = cb_grid.best_params_
cb_score = cb_grid.score(features_train_cb, target_train_cb)
print(cb_params)
print(cb_score)

**Тестирование наилучшей модели**

In [ ]:
%%time

#model = LogisticRegression(fit_intercept=True, 
#                                class_weight='balanced', 
#                                random_state=42,
#                                solver='liblinear')

#regression_parameters = {'C': [10]}

#Применяем GridSearchCV с кросс-валидацией
#regression_grid = GridSearchCV(model_lr, regression_parameters, scoring='f1', cv=3)
#regression_grid.fit(features_train, target_train)

#model.fit(features_train, target_train)

#lr_cv_score = pd.Series(cross_val_score(model_lr, features_train, target_train, scoring='f1', cv=3)).mean()
#predictions_test = model.predict(features_test)
#rmse = mean_squared_error(target_test, predictions_test)**0.5
#regression_f1 = round(f1_score(target_test, predictions_test), 3) 
#print(regression_f1)
#print('Среднее качество модели логистической регрессии на кросс-валидации', lr_cv_score)
#print('RMSE:', rmse)

In [ ]:
predictions_test = model_lr.best_estimator_.predict(features_test)
regression_f1 = round(f1_score(target_test, predictions_test), 3) 

In [ ]:
model = model_lr
words = TfidfVectorizer.get_feature_names_out().tolist()
weights = model.coef_.tolist()[0]

In [ ]:
feature_importance = pd.DataFrame(data=weights, index=words, columns=['coef'])
feature_importance.sort_values(by='coef',ascending=False).head(25).plot(kind='bar', title='top 25 most toxic words')

Проверим модель на адекватность

In [ ]:
#pred_previous = target_test.shift()
#pred_previous.iloc[0] = target_train.iloc[-1]
#rmse = mean_squared_error(target_test, pred_previous)**0.5

**Выводы**

В ходи исследования нами были обучены три модели, а именно Логистическая регрессия, случайный лес и CatBoost, а также определили для каждой модели оптимальные гипермараметры для кросс-валидации. В качестве тестирования модели мы выбрали модель логистической регрессии, поскольку она показала метрику качества f1 0.76, что соответствует условию задачи, а также RMSE 0,23.

## Выводы

1) В ходе загрузки и подготовки данных к обучению мы их загрузили, проверили на пропуски и дубликаты, а также удалили лишний столбец `Unnamed: 0`. Мы выделили целевой признак `lemm_text`. Также мы использовали TfidfVectorizer для векторизации текстов. Затем мы разделили данные на обучающую и тестовую выборки в соотношении 4 к 1.

2) В ходе обучения моделей мы выяснили что наилушей моделью для исследования подойдёт модель логистической регрессии, поскольку оставльные модели, а именно Случайный лес и CatBoost показали недостаточные f1-метрики для иследования, а именно 0,61 и 0.70 соответственно. Все модели прошли проверку на адекватность по сравнению с консантной моделью, где значение f1-меры 0.18.

**Bert**

In [ ]:
try:
    emb_toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
except:
    emb_toxic_comments = pd.read_csv('C:\\Data\\toxic_comments.csv')
    

In [ ]:
#emb_toxic_comments = emb_toxic_comments.sample(400).reset_index(drop=True)

#tokenizer = transformers.BertTokenizer(
#    vocab_file='/datasets/ds_bert/vocab.txt')

#tokenized = emb_toxic_comments['text'].apply(
#    lambda x: tokenizer.encode(x, add_special_tokens=True))

#max_len = 0
#for i in tokenized.values:
#    if len(i) > max_len:
#        max_len = len(i)

#padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

#attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
model = transformers.AutoModel.from_pretrained('unitary/toxic-bert')
tokenizer = transformers.AutoTokenizer.from_pretrained('unitary/toxic-bert')
features = np.concatenate(embeddings)
regression_grid = GridSearchCV(model_lr, regression_parameters, scoring='f1', cv=3, error_score='raise')
regression_grid.fit(features_train_new, target_train_new)

In [ ]:
#display(emb_toxic_comments['text'])

In [ ]:
#config = transformers.BertConfig.from_json_file(
#    '/datasets/ds_bert/bert_config.json')
#model = transformers.BertModel.from_pretrained(
#    '/datasets/ds_bert/rubert_model.bin', config=config)

In [ ]:
#batch_size = 101
#embeddings = []
#for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
#        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
#        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
#        with torch.no_grad():
#            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
#        embeddings.append(batch_embeddings[0][:,0,:].numpy())